# TensorFlow 2.0

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Hyperparameter Tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (32, 32, 3)
num_classes = 10

## Preprocess

In [3]:
import pandas as pd

In [4]:
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

In [5]:
train_df.head()

,path,class_name
0,C:/Users/신상훈/Desktop/새 폴더/dataset/cifar/train\...,frog
1,C:/Users/신상훈/Desktop/새 폴더/dataset/cifar/train\...,automobile
2,C:/Users/신상훈/Desktop/새 폴더/dataset/cifar/train\...,frog
3,C:/Users/신상훈/Desktop/새 폴더/dataset/cifar/train\...,frog
4,C:/Users/신상훈/Desktop/새 폴더/dataset/cifar/train\...,ship


In [6]:
test_df.head()

,path,class_name
0,C:/Users/신상훈/Desktop/새 폴더/dataset/cifar/test\0...,cat
1,C:/Users/신상훈/Desktop/새 폴더/dataset/cifar/test\1...,dog
2,C:/Users/신상훈/Desktop/새 폴더/dataset/cifar/test\1...,airplane
3,C:/Users/신상훈/Desktop/새 폴더/dataset/cifar/test\1...,ship
4,C:/Users/신상훈/Desktop/새 폴더/dataset/cifar/test\1...,deer


In [7]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.3,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_dataframe(
        train_df,
        x_col='path',
        y_col='class_name',
        target_size=input_shape[:2],
        batch_size=batch_size
)

validation_generator = test_datagen.flow_from_dataframe(
        test_df,
        x_col='path',
        y_col='class_name',
        target_size=input_shape[:2],
        batch_size=batch_size
)

Found 50000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


## Build Model

In [9]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [12]:
batch_size

32

In [13]:
len(train_df)//32

1562

In [14]:
len(train_generator)

1563

In [11]:
model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=1,
        validation_data=validation_generator,
        validation_steps=len(validation_generator))

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


1563/1563 [==============================] - 173s 110ms/step - loss: 2.0303 - accuracy: 0.2328 - val_loss: 1.5286 - val_accuracy: 0.4338
